# Abgabe 3 Mark Nagengast Porro, Tim Schwabe

In [224]:
import numpy as np

## Aufgabe 1

In [225]:
def RELU(x):
    return max(0,x)

In [226]:
def sigmoid(x):
    return 1/(1+(np.math.e)**(-x))

In [227]:
def createTrainingData(batch_size):

    data = np.array([[0,0],
                     [0,1],
                     [1,0],
                     [1,1]])
    truth = np.array([[0],[1],[1],[0]])
    
    training_data = []
    target = []
    while(batch_size > 0):
        index = np.random.randint(4)
        training_data.append(data[index])
        target.append(truth[index])
        batch_size -= 1
    return np.array(training_data), np.array(target)

train, target = createTrainingData(10)
#print(train)
#print(target)

## Aufgabe 2

In [228]:
class NN():

    def __init__(self, training_data, target_data,hidden_layers):

        self.training_data = training_data
        self.target_data = target_data
        self.z_list = []

        self.weights = [np.random.uniform(size=(2,4))]
        while(hidden_layers > 1):
            self.weights.append(np.random.uniform(size=(4,4)))
            hidden_layers -=1
        self.weights.append(np.random.uniform(size=(1,4)))


    def ReLU(self,x):
        return x * (x > 0) 
    
    def bcel(self, y, y_hat):
        return -(y*np.log(y_hat) + (1-y) * np.log(1-y_hat))
    
    def drv_loss_start(self, y, y_hat):
        return 1 / (1-y_hat) if y == 0 else -1 / y_hat

    def drv_loss(self, y, y_hat):
        return y_hat-y
    
    def drv_sigmoid(self, z):
        return sigmoid(z) * (1-sigmoid(z))
    
    def drv_ReLU(self, z):
        return 1 * (z > 0)

    def sigmoid(self, x):
        return 1/(1+(np.e)**(-np.array(x)))

    def backward(self, x, y, y_hat, i):
        step_size = 0.1
        self.weights[1] = self.weights[1] - step_size * self.drv_loss(y, y_hat) * self.ReLU(self.z_list[i])
        self.weights[0] = self.weights[0] - step_size * self.drv_loss(y, y_hat) * self.weights[1] * self.drv_ReLU(np.array(self.z_list[i])) * np.reshape(x, (2,1))


    def forward(self, x):
        for i in range(len(self.weights)-1):
            product = np.dot(x, self.weights[i])
            self.z_list.append(product)
            x = self.ReLU(np.array(product))
        return self.sigmoid(np.dot(self.weights[-1], x))
    
    def execute(self,x):
        return np.round(self.forward(x)[0])
    
    def accuracy(self, y, y_hats):
        counter = 0
        for i in range(len(y_hats)):
            print(f"Comparing {y_hats[i]} (rounded) and {y[i]}")
            counter += 1 if np.round(y_hats[i]) == y[i] else 0
        return counter / len(y_hats)

    def training(self):
        iter = 1
        while iter < 100:
            y_hats = []
            print(f"Iteration {iter}")
            for i in range(len(self.training_data)):
                y_hats.append(self.forward(self.training_data[i]))
            for i in range(len(self.training_data)):
                self.backward(self.training_data[i], self.target_data[i], y_hats[i], i)
            print(self.weights)

            print("BCEL across batch:")
            print(self.bcel(np.array(self.target_data), np.array(y_hats)))
            print("Accuracy across batch:")
            acc = self.accuracy(np.array(self.target_data), np.array(y_hats))
            print(f"{acc*100}%")
            if acc == 1: break
            iter += 1

        

In [229]:
test = NN(train,target,1)
test.training()


Iteration 1
[array([[ 0.06248662,  0.65912246,  0.68746053, -0.00419764],
       [ 0.23523894,  0.86780939,  0.56379821,  0.3105283 ]]), array([[0.01231271, 0.54689506, 0.47404395, 0.3122558 ]])]
BCEL across batch:
[[0.33079554]
 [0.33079554]
 [0.69314718]
 [2.32667648]
 [0.69314718]
 [0.33079554]
 [0.69314718]
 [0.33079554]
 [2.32667648]
 [0.33079554]]
Accuracy across batch:
Comparing [0.71835203] (rounded) and [1]
Comparing [0.71835203] (rounded) and [1]
Comparing [0.5] (rounded) and [0]
Comparing [0.90238035] (rounded) and [0]
Comparing [0.5] (rounded) and [0]
Comparing [0.71835203] (rounded) and [1]
Comparing [0.5] (rounded) and [0]
Comparing [0.71835203] (rounded) and [1]
Comparing [0.90238035] (rounded) and [0]
Comparing [0.71835203] (rounded) and [1]
80.0%
Iteration 2
[array([[6.53180501e-02, 6.76151606e-01, 7.01573530e-01, 2.16733193e-04],
       [2.38289803e-01, 8.00152941e-01, 5.01739121e-01, 2.66727480e-01]]), array([[-0.02682656,  0.38899653,  0.36878362,  0.25148961]])]
BC